<a href="https://colab.research.google.com/github/Simo01sp/Repository_Biavasco/blob/main/MHCH_Canva__primi3turni_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import numpy as np
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util
import nltk

nltk.download('vader_lexicon')
nltk.download('punkt')

# ------------------------------------------------------------
# REGEX
# ------------------------------------------------------------
FRUSTRATION_RE = re.compile(
    r"(angry|furious|annoyed|unacceptable|ridiculous|frustrat|"
    r"again|yesterday|same|late|delay|waiting|"
    r"doesn'?t work|not working|broken|issue|problem)",
    re.IGNORECASE
)

HUMAN_TRIGGER_RE = re.compile(
    r"(transfer.*human|transfer.*agent|transfer.*person|"
    r"talk.*human|talk.*agent|talk.*person|"
    r"speak.*human|speak.*agent|speak.*person|"
    r"i want to talk to someone|i want to talk to a person|"
    r"i want to talk to someone real|i want to talk to a real person|"
    r"talk to someone|talk to a person|"
    r"speak to someone|speak to a person|"
    r"human agent now|agent now|human please|agent please|"
    r"i need a human|i need an agent|real person|real human|"
    r"let me talk to|let me speak to|someone real|human support)",
    re.IGNORECASE
)

STRONG_FRUSTRATION_RE = re.compile(
    r"(ridiculous|unacceptable|this is ridiculous|this is unacceptable)",
    re.IGNORECASE
)

# REJECTION (forte contestazione)
REJECTION_RE = re.compile(
    r"(can't accept|cannot accept|dont accept|don't accept|"
    r"this doesn't make sense|doesnt make sense|this does not make sense|"
    r"this is wrong|that is wrong|not acceptable|not okay|"
    r"nothing i can do|so there's nothing|so there is nothing|"
    r"i disagree)",
    re.IGNORECASE
)

# ------------------------------------------------------------
# READABILITY
# ------------------------------------------------------------
def simple_readability(text: str) -> float:
    words = text.split()
    n_words = len(words)
    n_sent = max(text.count(".") + text.count("!") + text.count("?"), 1)
    avg = n_words / n_sent
    return max(0, min(100, 100 - avg * 4))


# ============================================================
#                    HANDOFF PREDICTOR
# ============================================================
class HandoffPredictor:

    def __init__(self):
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2")
        self.sia = SentimentIntensityAnalyzer()

    # ------------------------------------------------------------
    # DIFFICULTY
    # ------------------------------------------------------------
    def compute_difficulty(self, text: str, context):

        text_low = text.lower().replace("’", "'").replace("‘", "'")

        # Override frasi che vanno sempre trasferite
        if any(p in text_low for p in [
            "don't want to wait", "cant someone", "can't someone", "dont want to wait"
        ]):
            return 0.95

        # Sentiment
        sent = self.sia.polarity_scores(text)["compound"]
        emotion_score = 1 - ((sent + 1) / 2)

        # Trigger forti
        if STRONG_FRUSTRATION_RE.search(text_low):
            return 1.0

        # Turn number
        current_turn = len(context) + 1
        turn_weight = min(1.0, 0.1 * current_turn)

        # Frustrazione lessicale base
        base_hits = FRUSTRATION_RE.findall(text)
        base_frust_score = min(len(base_hits) * 0.30, 1.0)

        # Parole "frustration"
        frust_words = ["frustrating", "frustration", "frustrated"]
        current_is_frustration = any(w in text_low for w in frust_words)
        previous_frustrations = sum(
            1 for t in context if any(w in t.lower() for w in frust_words)
        )

        frustration_score = base_frust_score

        if current_is_frustration:
            base = emotion_score * turn_weight

            if current_turn <= 2:
                repetition_weight = 0.15 if previous_frustrations == 0 else 0.25
            elif current_turn == 3:
                repetition_weight = 0.35
            elif 4 <= current_turn <= 6:
                repetition_weight = 0.5 if previous_frustrations == 0 else 0.65
            else:
                repetition_weight = 0.75 if previous_frustrations == 0 else 0.95

            frustration_score = max(frustration_score, min(base * repetition_weight, 1.0))

        # Stress / anxiety
        stress_words = ["stressed", "stress", "anxious", "anxiety", "overwhelmed"]
        if any(w in text_low for w in stress_words):
            stress_score = emotion_score * (0.4 + 0.6 * turn_weight)
            if current_turn >= 7:
                stress_score = max(stress_score, 0.70)
            frustration_score = max(frustration_score, min(stress_score, 1.0))

        # Emotional frustration (forte)
        emo_f_words = ["disappointed", "disappointing", "unfair",
                       "hopeless", "hassle", "upset", "sad"]  # ← "lost" rimosso

        if any(w in text_low for w in emo_f_words):
            emo_f_score = emotion_score * (0.5 + 0.5 * turn_weight)
            if current_turn >= 5:
                emo_f_score = max(emo_f_score, 0.45)
            if current_turn >= 7:
                emo_f_score = max(emo_f_score, 0.65)
            frustration_score = max(frustration_score, min(emo_f_score, 1.0))

        # PATCH 2 — soft emotional words
        soft_emotional_words = ["lost", "confused", "uncertain"]

        if any(w in text_low for w in soft_emotional_words):
            soft_emo_score = emotion_score * 0.35
            if current_turn >= 6:
                soft_emo_score = max(soft_emo_score, 0.45)
            frustration_score = max(frustration_score, soft_emo_score)

        # Worried
        worried_words = ["worried", "concerned", "unclear", "unsure"]
        if any(w in text_low for w in worried_words) and frustration_score < 0.45:
            return 0.40

        # Still repetition
        if "still" in text_low:
            reps = sum("still" in t.lower() for t in context[-3:])
            frustration_score += [0.03, 0.10, 0.25, 0.40][min(reps, 3)]
            frustration_score = min(frustration_score, 1.0)

        # Rejection forte
        if REJECTION_RE.search(text_low):
            if current_turn >= 6:
                return 0.85
            return 0.70

        # EARLY TURN CAP — turni 1–3
        if current_turn <= 3:
            frustration_score = min(frustration_score, 0.40)

        # Escalation tardi
        if current_turn >= 6 and (current_is_frustration or base_frust_score > 0):
            frustration_score = max(frustration_score, 0.55)

        # Sentiment troppo alto nei primi turni → limita
        if sent > -0.25 and current_turn <= 3:
            frustration_score = min(frustration_score, 0.35)

        # Length + complexity
        length_score = min(len(word_tokenize(text)) / 20, 1)
        readability = simple_readability(text)
        complexity_score = 1 - min(readability / 100, 1)

        # Direct human request
        if HUMAN_TRIGGER_RE.search(text_low):
            return 1.0

        score = (
            0.30 * emotion_score +
            0.40 * frustration_score +
            0.20 * complexity_score +
            0.10 * length_score
        )
        return round(min(score, 1), 2)

    # ------------------------------------------------------------
    # MATCHING SCORE
    # ------------------------------------------------------------
    def compute_matching(self, text, context, prev_scores):
        if not context:
            return 0.0, prev_scores

        emb_cur = self.embedder.encode(text, convert_to_tensor=True)
        emb_prev = self.embedder.encode(context[-4:], convert_to_tensor=True)

        sim = float(np.max(util.cos_sim(emb_cur, emb_prev).cpu().numpy()[0]))

        # Sensibilità aumentata
        if sim < 0.4:
            sim = sim ** 0.55 + 0.10

        frust_hits = len(FRUSTRATION_RE.findall(text))
        frust_boost = min(frust_hits * 0.30, 1)

        explicit = any(p in text.lower() for p in ["again", "still", "third time"])
        explicit_boost = 0.30 if explicit else 0.0

        prev_scores.append(sim)
        if len(prev_scores) > 5:
            prev_scores.pop(0)

        many = sum(s > 0.50 for s in prev_scores) >= 2
        repetition_boost = 0.30 if many else 0.0

        match = (
            0.50 * sim +
            0.35 * frust_boost +
            0.15 * explicit_boost +
            0.20 * repetition_boost
        )

        return round(min(match, 1), 2), prev_scores

    # ------------------------------------------------------------
    # FINAL DECISION
    # ------------------------------------------------------------
    def predict_turn(self, text, context, prev_scores,
                     threshold=0.45, direct_threshold=0.50):

        diff = self.compute_difficulty(text, context)
        match, prev_scores = self.compute_matching(text, context, prev_scores)

        # Forte ripetizione o alta similarità
        if match > direct_threshold:
            total = match
            label = "TRANSFERABLE"

        # Frustrazione molto forte
        elif diff > 0.70:
            total = diff
            label = "TRANSFERABLE"

        else:
            # NEW — EARLY TURN SAFETY RULE
            current_turn = len(context) + 1
            if current_turn <= 3 and diff < 0.60:
                return {
                    "text": text,
                    "difficulty": diff,
                    "matching": match,
                    "score": round(0.0, 2),
                    "label": "NORMAL"
                }, prev_scores

            total = 0.60 * diff + 0.40 * match
            label = "TRANSFERABLE" if total > threshold else "NORMAL"

        prev_scores = [float(np.real(v)) for v in prev_scores]

        return {
            "text": text,
            "difficulty": diff,
            "matching": match,
            "score": round(total, 2),
            "label": label
        }, prev_scores

    # ------------------------------------------------------------
    # PROCESSA INTERA CHAT
    # ------------------------------------------------------------
    def predict_conversation(self, turns):
        context = []
        prev_scores = []
        results = []

        for t in turns:
            res, prev_scores = self.predict_turn(t, context, prev_scores)
            results.append(res)
            context.append(t)

        return results


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import re

# Carica dataset
df = pd.read_csv("conversazioni_MHCH.csv")

# SCEGLI TU quale conversazione testare
row_index = 2# <---- CAMBIA QUESTO NUMERO
conv_text = df.loc[row_index, "Conversazione"]

In [ ]:
def extract_user_turns(conv_text):
    user_turns = []
    for line in str(conv_text).split("\n"):
        line = line.strip()
        if re.match(r"^(user|u|utente)\s*[:\-]", line, flags=re.IGNORECASE):
            msg = re.sub(r"^(user|u|utente)\s*[:\-]\s*", "", line, flags=re.IGNORECASE)
            user_turns.append(msg)
    return user_turns

conversation = extract_user_turns(conv_text)

In [ ]:
import nltk
nltk.download('punkt_tab')
hp = HandoffPredictor()

results = hp.predict_conversation(conversation)

print(f"\n💬 Conversation analysis — riga {row_index}")
print("-" * 80)

for i, r in enumerate(results, start=1):
    print(f"TURN {i} — USER: {r['text']}")
    print(f"→ Difficulty={r['difficulty']} | Matching={r['matching']} | Total={r['score']} → {r['label']}\n")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!



💬 Conversation analysis — riga 2
--------------------------------------------------------------------------------
TURN 1 — USER: What is an Appeal?
→ Difficulty=0.21 | Matching=0.0 | Total=0.0 → NORMAL

TURN 2 — USER: How do I file an appeal? It sounds complicated.
→ Difficulty=0.24 | Matching=0.34 | Total=0.0 → NORMAL

TURN 3 — USER: What if I miss the 7-day deadline?
→ Difficulty=0.27 | Matching=0.22 | Total=0.0 → NORMAL

TURN 4 — USER: That's not fair! I might not have all the documents ready in time.
→ Difficulty=0.25 | Matching=0.37 | Total=0.3 → NORMAL

TURN 5 — USER: But what if I don't agree with the evaluation but can't provide more documents?
→ Difficulty=0.41 | Matching=0.33 | Total=0.38 → NORMAL

TURN 6 — USER: This is really frustrating. I just want to challenge the evaluation!
→ Difficulty=0.54 | Matching=0.38 | Total=0.48 → TRANSFERABLE

TURN 7 — USER: What kind of documents do I need to upload?
→ Difficulty=0.27 | Matching=0.27 | Total=0.27 → NORMAL

TURN 8 — USER: Why